In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
cd ../src

/home/tviel/work/kaggle_rsna_lumbar_spine/src


/home/tviel/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [29]:
import os
import cv2
import json
import glob
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch.nn.functional as F

from collections import Counter
from tqdm.notebook import tqdm


pd.set_option('display.width', 500)
pd.set_option('max_colwidth', 100)

In [30]:
from params import *

from data.preparation import prepare_data

from util.plots import *
from util.logger import Config
from util.torch import load_model_weights

from model_zoo.models_seg import define_model

In [33]:
# from boto3.session import Session
# import boto3
# import os



# BUCKET_NAME = 'kaggle-rsna24'
# ACCESS_KEY = ''
# SECRET_KEY = ''

# session = Session(aws_access_key_id=ACCESS_KEY, aws_secret_access_key=SECRET_KEY)
# s3 = session.resource('s3')
# your_bucket = s3.Bucket(BUCKET_NAME)

# fns = []
# for s3_file in your_bucket.objects.all():
#     fns += [s3_file.key]


# #     target_folder = '/'.join(fn.split('/')[:-1]).replace('models/cfg_ch_1c8d/','')
    
#     target_file = fn.split('/')[-1]
# #     os.makedirs(target_folder, exist_ok=True)
# #     print(fn, ' -> ', target_folder + '/' + target_file)
#     s3.Bucket(BUCKET_NAME).download_file(fn, target_file)

# for fn in tqdm(fns):
#     if "train_folded_v1" in fn:
#         print(fn)
#         target_file = fn.split('/')[-1]
#         s3.Bucket(BUCKET_NAME).download_file(fn, target_file)

In [5]:
df = prepare_data(DATA_PATH)

In [6]:
df_train = pd.read_csv(DATA_PATH + "train.csv")

In [7]:
df_train[df_train[df_train.columns[:6]].isna().sum(1) > 0]

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
1378,3008676218,NaN,NaN,NaN,NaN,NaN,Normal/Mild,Severe,Moderate,Moderate,...,Normal/Mild,Moderate,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Severe,Normal/Mild,Normal/Mild


In [8]:
len(df_train), len(df.study_id.unique())

(1975, 1975)

In [9]:
import re
df['condition'] = df['condition'].apply(lambda d: d if isinstance(d, list) else [])

df['condition'] = df['condition'].apply(lambda x: [re.sub('Neural Foraminal Narrowing', 'NFN', u) for u in x])
df['condition'] = df['condition'].apply(lambda x: [re.sub('Spinal Canal Stenosis', 'SCS', u) for u in x])
df['condition'] = df['condition'].apply(lambda x: [re.sub('Subarticular Stenosis', 'SS', u) for u in x])
df['condition'] = df['condition'].apply(lambda x: [re.sub('Left', 'L', u) for u in x])
df['condition'] = df['condition'].apply(lambda x: [re.sub('Right', 'R', u) for u in x])

In [10]:
df[df['weighting'] == "T1"].condition.apply(len).value_counts() / (len(df) / 3)

condition
10    0.935653
5     0.006673
0     0.000477
15    0.000477
9     0.000477
Name: count, dtype: float64

In [11]:
df[df['weighting'] == "T2/STIR"].condition.apply(len).value_counts()

condition
5    1899
3      38
4      35
1       1
0       1
Name: count, dtype: int64

In [12]:
df_ = pd.read_csv(DATA_PATH + "train_label_coordinates.csv")
df_ = df_[df_['condition'].apply(lambda x: 'Neural Foraminal Narrowing' in x)].reset_index(drop=True)
df_["side"] = df_['condition'].apply(lambda x: x.split()[0])

frames = pd.read_csv('../input/df_frames.csv')
frames["frames"] = frames["frames"].apply(eval)
df_ = df_.merge(frames, how="left")
df_['ordered'] = df_['frames'].apply(lambda x: x[0] < x[-1])

dfg = df_[["study_id", "series_id", "side", "instance_number", "ordered"]].groupby(["study_id", "series_id", "side"]).mean()
dfg['ordered'] = dfg['ordered'] > 0.5
dfg = dfg.reset_index().groupby(["study_id", "series_id"])

In [13]:
df['x'] = df['coords'].apply(lambda x: [int(u[0]) for u in x] if isinstance(x, np.ndarray) else -1)

In [14]:
df[df['study_id'] == 3637444890]

,study_id,series_id,series_description,orient,weighting,img_path,condition,level,coords,x
5330,3637444890,1590743005,Axial T2,Axial,T2,../input/npy/3637444890_1590743005.npy,"[L SS, R SS, L SS, R SS, L SS, R SS]","[L3/L4, L3/L4, L4/L5, L4/L5, L5/S1, L5/S1]","[[3.0, 176.13040629095676, 175.72739187418088], [3.0, 153.80185979971387, 173.5050071530758], [1...","[3, 3, 10, 11, 16, 17]"
5331,3637444890,3671685092,Axial T2,Axial,T2,../input/npy/3637444890_3671685092.npy,"[L SS, R SS, L SS, R SS]","[L1/L2, L1/L2, L2/L3, L2/L3]","[[9.0, 183.26015727391876, 175.72739187418088], [9.0, 147.85050071530756, 174.87839771101574], [...","[9, 9, 17, 17]"
5332,3637444890,3892989905,Sagittal T2/STIR,Sagittal,T2/STIR,../input/npy/3637444890_3892989905.npy,[],NaN,NaN,-1
5333,3637444890,3951475160,Sagittal T1,Sagittal,T1,../input/npy/3637444890_3951475160.npy,"[R NFN, R NFN, R NFN, R NFN, R NFN, SCS, SCS, SCS, SCS, SCS, L NFN, L NFN, L NFN, L NFN, L NFN]","[L4/L5, L5/S1, L1/L2, L2/L3, L3/L4, L1/L2, L2/L3, L3/L4, L4/L5, L5/S1, L1/L2, L2/L3, L3/L4, L4/L...","[[3.0, 194.8471074380165, 224.26446280991732], [3.0, 201.72314049586777, 259.702479338843], [4.0...","[3, 3, 4, 4, 4, 7, 7, 7, 7, 7, 10, 11, 11, 11, 11]"


In [26]:
for s in tqdm(df.series_id.unique()):
    dfs = df[df['series_id'] == s]

    if dfs["weighting"].values[0] != "T1":
        continue

    xs = dfs['x'].values[0]
    sides = [l[0] for l in dfs['condition'].values[0]]

    dfc_ = pd.DataFrame({'x': xs, 's': sides})
    dfc = dfc_.groupby("s").mean()
    
    try:
        if not dfc["x"]["R"] < dfc["x"]["L"]:
            # print(s)
            display(dfc_)
    except KeyError:
        pass
    # display(dfc)
    # break

  0%|          | 0/6294 [00:00<?, ?it/s]

,x,s
0,10,R
1,10,R
2,9,R
3,8,R
4,8,R
5,4,L
6,3,L
7,3,L
8,3,L
9,1,L


,x,s
0,11,L
1,11,L
2,11,L
3,11,L
4,11,L
5,11,R
6,11,R
7,11,R
8,11,R
9,11,R


,x,s
0,3,L
1,3,L
2,3,R
3,3,R
4,4,L
5,4,L
6,4,L
7,4,R
8,4,R
9,4,R


,x,s
0,2,L
1,2,L
2,3,L
3,3,L
4,3,L
5,11,R
6,11,R
7,11,R
8,12,R
9,12,R


,x,s
0,9,L
1,9,L
2,9,L
3,9,R
4,9,R
5,9,R
6,8,L
7,8,L
8,8,R
9,8,R


,x,s
0,11,L
1,11,L
2,11,L
3,11,L
4,11,R
5,11,R
6,11,R
7,11,R
8,10,L
9,10,R


,x,s
0,6,L
1,6,R
2,5,L
3,5,L
4,5,R
5,5,R
6,4,L
7,4,L
8,4,R
9,4,R


In [ ]:
labels = pd.read_csv(DATA_PATH + "train_label_coordinates.csv")
labels[labels['series_id'] == 1032434193]

In [ ]:
dfs = df[df['series_id'] == 1032434193]
dfs

In [ ]:
sorted(os.listdir(DATA_PATH + f"train_images/{dfs.study_id.values[0]}/{dfs.series_id.values[0]}"), key=lambda x: int(x.split('.')[0]))

In [ ]:
df_train[df_train['study_id'] == 4003253].T

In [ ]:
df_train

Done ! 